# Normalize crops across image space

In [1]:
from functions.crop_image import random_crop
from functions.load_all_data import load_data_by_color, load_labeled_data
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np
import os

In [2]:
purple_png_list, purple_img_paths, purple_mask_colls = load_data_by_color(color="Purple")
def_png_list, def_img_paths, def_mask_colls = load_data_by_color(color="Default")

In [ ]:
training_samples = []
training_labels = []

test_samples = []
test_labels = []

for i in range(1000):
    # take 100 random crops of random images in the purple and default sets
    # take samples from first half of respective data sets
    
    def_idx = np.random.randint(0, len(def_img_paths))
    purp_idx = np.random.randint(0, len(purple_img_paths))
    
    def_img = io.imread(def_img_paths[def_idx])
    purp_img = io.imread(purple_img_paths[purp_idx])
    
    def_img = random_crop(def_img)
    purp_img = random_crop(purp_img)
    
    training_samples.append(def_img)
    training_samples.append(purp_img)
    

    training_labels.append(0.)
    training_labels.append(1.)

    
for i in range(10):
    # test data
    # take 10 random crops of random images in the other half of the image sets

    def_idx = np.random.randint(0, len(def_img_paths))
    purp_idx = np.random.randint(0, len(purple_img_paths))
    
    def_img = io.imread(def_img_paths[def_idx])
    purp_img = io.imread(purple_img_paths[purp_idx])
    
    def_img = random_crop(def_img)
    purp_img = random_crop(purp_img)
    
    
    test_samples.append(def_img)
    test_samples.append(purp_img)
    
    test_labels.append(0.)
    test_labels.append(1.)


In [ ]:
count = 0
for i in training_samples:
    io.imshow(i)
    plt.figure()

    if count == 3: 
        break
    
    count += 1

In [ ]:
training_samples = np.array(training_samples)
training_labels = np.array(training_labels)

test_samples = np.array(test_samples)
test_labels = np.array(test_labels)

training_samples = training_samples / 255.
test_samples = test_samples / 255.

In [ ]:
from functions.tf_config import config_gpu

config_gpu(5, True)

In [ ]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, input_shape=(64, 64, 3), activation='relu', strides=(2,2)),
    keras.layers.Conv2D(16, 3, activation='relu', strides=(2,2)),
    keras.layers.Conv2D(8, 3, activation='relu', strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(2)
])

In [ ]:
predictions = model(training_samples).numpy()
predictions

In [ ]:
tf.nn.softmax(predictions).numpy()

In [ ]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(training_labels, predictions).numpy()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=.001)
model.compile(optimizer=opt,
             loss=loss_fn,
             metrics=['accuracy'])

In [ ]:
model.fit(training_samples, training_labels, epochs=5, batch_size=8)

In [ ]:
model.evaluate(test_samples, test_labels, verbose=2)